In [12]:
# https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/

# FYI, 1) Prophet's name has changed from fbprophet to prophet. So anywhere instructions say fbprophet use prophet 
# 2) it only runs on versions of python less than 3.9. If you're like me, I have 3.7 installed in PythonData kernal, so I installed Prophet there using the following instructions. 
# https://medium.com/data-folks-indonesia/installing-fbprophet-prophet-for-time-series-forecasting-in-jupyter-notebook-7de6db09f93e

In [52]:
# check prophet version
import prophet
# print version number
print('Prophet %s' % prophet.__version__)

from pandas import read_csv
from matplotlib import pyplot

from pandas import to_datetime

from prophet import Prophet

from pandas import DataFrame

import pandas as pd
import numpy as np

Prophet 1.0


In [69]:
# training percentage of dataset data

train_perc = 0.80

# daily seasonality

seasonality_b = False

# remove zeros

zeros_b = False  # True = remove zero values from dataset, False = keep zeros in dataset

In [3]:
# Load metrics from CVS files

#df_dailysteps=pd.read_csv("Resources\steps_p01.csv",index_col='Dates',parse_dates=True)

#df_steps=pd.read_csv("final_merged_all_participants/steps.csv")
#df_active_duration=pd.read_csv("final_merged_all_participants/active_duration.csv")
#df_active_heart_rate=pd.read_csv("final_merged_all_participants/active_heart_rate.csv")
#df_calories=pd.read_csv("final_merged_all_participants/calories.csv")
#df_distance=pd.read_csv("final_merged_all_participants/distance.csv")
#df_resting_heartrate=pd.read_csv("final_merged_all_participants/resting_heartrate.csv")
#df_sleep=pd.read_csv("final_merged_all_participants/sleep.csv")
#df_participant_overview=pd.read_csv("final_merged_all_participants/participant_overview.csv")

#df_sleep.rename(columns = {"Date_Of_Sleep": "Dates"}, inplace = True)

# Drop the label 'time'
#df_active_heart_rate = df_active_heart_rate.drop(columns="Time")

In [54]:
# Load metrics from Database

import psycopg2
from psycopg2 import Error
from config import db_password
from config import user

try:
    # Connect to an existing database
    connection = psycopg2.connect(user= user,
                                  password=db_password,
                                  host="ec2-3-231-82-226.compute-1.amazonaws.com",
                                  port="5432",
                                  database="derfg6csuck3l8")
    # Create a cursor to perform database operations
    cursor = connection.cursor()
    
     # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    
# 1) Executing a SQL query for participant details
    cursor.execute("SELECT * FROM participants_details;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_participant_overview = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 2) Executing a SQL query for steps details
    cursor.execute("SELECT * FROM steps;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_steps = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 3) Executing a SQL query for active_duration details
    cursor.execute("SELECT * FROM active_duration;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_active_duration = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 4) Executing a SQL query for v_avg_active_heartrate details
    cursor.execute("SELECT * FROM v_avg_active_heartrate;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_active_heart_rate = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 5) Executing a SQL query for calories details
    cursor.execute("SELECT * FROM calories;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_calories = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 6) Executing a SQL query for distance details
    cursor.execute("SELECT * FROM distance;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_distance = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 7) Executing a SQL query for resting_heartrate details
    cursor.execute("SELECT * FROM resting_heartrate;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_resting_heartrate = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")
    
# 8) Executing a SQL query for sleep details
    cursor.execute("SELECT * FROM sleep;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_sleep = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")
    
    
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'yeujsrmjhunnke', 'channel_binding': 'prefer', 'dbname': 'derfg6csuck3l8', 'host': 'ec2-3-231-82-226.compute-1.amazonaws.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

PostgreSQL connection is closed


In [55]:
# keep only particpant Ids

df_participant_IDs = df_participant_overview["participant_id"]

In [56]:
def parser(df):
    df.dropna()
    df = df.drop(["participant_id"],axis=1)
    if "S.No." in df:
        df = df.drop(["S.No."],axis=1)
    df.columns = ['ds', 'y']
    df['ds']= to_datetime(df['ds'])
    
    return df

In [57]:
def evaluate_model(predictions, test, perf_mape):
    
    
    if perf_mape:
        test = test.reset_index()
        predictions = predictions.reset_index()
        mape = round(100 * np.mean(np.abs(predictions['yhat'] - test['y'])/np.abs(test['y'])),2)  # MAPE.ipynb
    else:
        mape = "No Data"
    return mape

In [70]:
active_duration_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Active_Duration_in_mins'])
steps_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Minutes_Asleep'])
active_heart_rate_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Avg_ActiveHeartRate'])
distance_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Distance'])
calories_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Calories'])
resting_heartrate_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Resting_beats_per_min'])
sleep_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Minutes_Asleep'])
mape_df = pd.DataFrame(columns = ['Participant_ID', 'steps', 'active_duration', 'active_heart_rate', 'distance', 'calories', 'resting_heartrate', 'sleep', 'Training_Percentage', 'Daily_Seasonality'])

In [71]:
for id in df_participant_IDs:
    
    print(id)
# loop through Participant IDs

    if zeros_b:
        df_steps_X = df_steps.loc[(df_steps["participant_id"] == id) & (df_steps["steps"] > 0), :]
        df_active_duration_X = df_active_duration.loc[(df_active_duration["participant_id"] == id) & (df_active_duration["active_duration_in_mins"] > 0), :]
        df_active_heart_rate_X = df_active_heart_rate.loc[(df_active_heart_rate["participant_id"] == id) & (df_active_heart_rate["avg"] > 0), :]
        df_distance_X = df_distance.loc[(df_distance["participant_id"] == id) & (df_distance["distance"] > 0), :]
        df_calories_X = df_calories.loc[(df_calories["participant_id"] == id) & (df_calories["calories"] > 0), :]
        df_resting_heartrate_X = df_resting_heartrate.loc[(df_resting_heartrate["participant_id"] == id) & (df_resting_heartrate["resting_beats_per_min"] > 0), :]
        df_sleep_X = df_sleep.loc[(df_sleep["participant_id"] == id) & (df_sleep["minutes_asleep"] > 0), :]
    else:
        df_steps_X = df_steps.loc[df_steps["participant_id"] == id, :]
        df_active_duration_X = df_active_duration.loc[df_active_duration["participant_id"] == id, :]
        df_active_heart_rate_X = df_active_heart_rate.loc[df_active_heart_rate["participant_id"] == id, :]
        df_distance_X = df_distance.loc[df_distance["participant_id"] == id, :]
        df_calories_X = df_calories.loc[df_calories["participant_id"] == id, :]
        df_resting_heartrate_X = df_resting_heartrate.loc[df_resting_heartrate["participant_id"] == id, :]
        df_sleep_X = df_sleep.loc[df_sleep["participant_id"] == id, :]
    
    steps_b = True
    active_duration_b = True
    active_heart_rate_b = True
    distance_b = True
    calories_b = True
    resting_heartrate_b = True
    sleep_b = True
    
    #test to see if dataset has any data for participant ID
    if df_steps_X.empty:
        steps_b = False
    
    if df_active_duration_X.empty:
        active_duration_b = False
        
    if df_active_heart_rate_X.empty:
        active_heart_rate_b = False
        
    if df_distance_X.empty:
        distance_b = False
        
    if df_calories_X.empty:
        calories_b = False
        
    if df_resting_heartrate_X.empty:
        resting_heartrate_b = False
    
    if df_sleep_X.empty:
        sleep_b = False
            
    # parse data for model    
    df_steps_X = parser(df_steps_X)
    df_active_duration_X = parser(df_active_duration_X)
    df_active_heart_rate_X = parser(df_active_heart_rate_X)
    df_distance_X = parser(df_distance_X)
    df_calories_X = parser(df_calories_X)
    df_resting_heartrate_X = parser(df_resting_heartrate_X)
    df_sleep_X = parser(df_sleep_X)

# set train and test data
    train_size = int(len(df_steps_X) * abs(train_perc))
    df_steps_test, df_steps_train = df_steps_X[0:train_size], df_steps_X[train_size:]

    train_size = int(len(df_active_duration_X) * abs(train_perc))
    df_active_duration_test, df_active_duration_train = df_active_duration_X[0:train_size], df_active_duration_X[train_size:]

    train_size = int(len(df_active_heart_rate_X) * abs(train_perc))
    df_active_heart_rate_test, df_active_heart_rate_train = df_active_heart_rate_X[0:train_size], df_active_heart_rate_X[train_size:]
    
    train_size = int(len(df_distance_X) * abs(train_perc))
    df_distance_test, df_distance_train = df_distance_X[0:train_size], df_distance_X[train_size:]
    
    train_size = int(len(df_calories_X) * abs(train_perc))
    df_calories_test, df_calories_train = df_calories_X[0:train_size], df_calories_X[train_size:]

    train_size = int(len(df_resting_heartrate_X) * abs(train_perc))
    df_resting_heartrate_test, df_resting_heartrate_train = df_resting_heartrate_X[0:train_size], df_resting_heartrate_X[train_size:]
    
    train_size = int(len(df_sleep_X) * abs(train_perc))
    df_sleep_test, df_sleep_train = df_sleep_X[0:train_size], df_sleep_X[train_size:]
    
    
# define the model, fit and test
    steps_test_forecast = ""
    active_duration_test_forcast = ""
    active_heart_rate_test_forcast = ""
    distance_test_forcast = ""
    calories_test_forcast = ""
    resting_heartrate_test_forcast = ""
    sleep_test_forcast = ""
    
    if steps_b:
        model_steps = Prophet(daily_seasonality=seasonality_b)
        model_steps.fit(df_steps_train)
        steps_test_forecast = model_steps.predict(df_steps_test)
        
        future = model_steps.make_future_dataframe(periods=365)   # predict future time series
        steps_train_predict = model_steps.predict(future)
        
#        steps_forcast_df = steps_forcast_df.append(steps_train_predict,
#                                                  ignore_index=True)        
        
    if active_duration_b:
        model_active_duration = Prophet(daily_seasonality=seasonality_b)
        model_active_duration.fit(df_active_duration_train)
        active_duration_test_forcast = model_active_duration.predict(df_active_duration_test)
        
        future = model_active_duration.make_future_dataframe(periods=365) # predict future time series
        active_duration_train_predict = model_active_duration.predict(future)

#        active_duration_forcast_df = active_duration_forcast_df.append(active_duration_train_predict,
#                                                  ignore_index=True)        
        
    if active_heart_rate_b:
        model_active_heart_rate = Prophet(daily_seasonality=seasonality_b)
        model_active_heart_rate.fit(df_active_heart_rate_train)
        active_heart_rate_test_forcast = model_active_heart_rate.predict(df_active_heart_rate_test)
        
        future = model_active_heart_rate.make_future_dataframe(periods=365)    # predict future time series
        active_heart_rate_train_predict = model_active_heart_rate.predict(future)
        
 #       active_heart_rate_forcast_df = active_heart_rate_forcast_df.append(active_heart_rate_train_predict,
#                                                  ignore_index=True)
        
    if distance_b:
        model_distance = Prophet(daily_seasonality=seasonality_b)
        model_distance.fit(df_distance_train)
        distance_test_forcast = model_distance.predict(df_distance_test)
        
        future = model_distance.make_future_dataframe(periods=365)     # predict future time series
        distance_train_predict = model_distance.predict(future)
        
#        distance_forcast_df = distance_forcast_df.append(distance_train_predict,
 #                                                 ignore_index=True)
        
    if calories_b:
        model_calories = Prophet(daily_seasonality=seasonality_b)
        model_calories.fit(df_calories_train)
        calories_test_forcast = model_calories.predict(df_calories_test)
        
        future = model_calories.make_future_dataframe(periods=365)    # predict future time series
        calories_train_predict = model_calories.predict(future)

 #       calories_forcast_df = calories_forcast_df.append(calories_train_predict,
 #                                                 ignore_index=True)
        
    if resting_heartrate_b:
        model_resting_heartrate = Prophet(daily_seasonality=seasonality_b)
        model_resting_heartrate.fit(df_resting_heartrate_train)
        resting_heartrate_test_forcast = model_resting_heartrate.predict(df_resting_heartrate_test)
        
        future = model_resting_heartrate.make_future_dataframe(periods=365)   # predict future time series
        resting_heartrate_train_predict = model_resting_heartrate.predict(future)
        
 #       resting_heartrate_forcast_df = resting_heartrate_forcast_df.append(resting_heartrate_train_predict,
 #                                                 ignore_index=True)
     
    if sleep_b:
        model_sleep = Prophet(daily_seasonality=seasonality_b)
        model_sleep.fit(df_sleep_train)
        sleep_test_forcast = model_sleep.predict(df_sleep_test)
        
        future = model_sleep.make_future_dataframe(periods=365)    # predict future time series
        sleep_train_predict = model_sleep.predict(future)
        
#        sleep_forcast_df = sleep_forcast_df.append(sleep_train_predict,
#                                                  ignore_index=True)
    
#    df3 = df1.append(df2, ignore_index=True)

    # print(evaluate_model(steps_test_forecast, df_steps_test, steps_b))
    # print(evaluate_model(active_duration_test_forcast, df_active_duration_test, active_duration_b))
    
    mape_df = mape_df.append({'Participant_ID' : id,
                    'steps' : evaluate_model(steps_test_forecast, df_steps_test, steps_b),
                    'active_duration' : evaluate_model(active_duration_test_forcast, df_active_duration_test, active_duration_b),
                    'active_heart_rate' : evaluate_model(active_heart_rate_test_forcast, df_active_heart_rate_test, active_heart_rate_b),
                    'distance' : evaluate_model(distance_test_forcast, df_distance_test, distance_b),
                    'calories' : evaluate_model(calories_test_forcast, df_calories_test, calories_b),
                    'resting_heartrate' : evaluate_model(resting_heartrate_test_forcast, df_resting_heartrate_test, resting_heartrate_b),
                    'sleep' : evaluate_model(sleep_test_forcast, df_sleep_test, sleep_b),
                    'Training_Percentage': train_perc*100,
                    'Daily_Seasonality': seasonality_b},
                    ignore_index=True)
    
    
    # print(mape_df.head())
    #break
# predict future time series

#    future = model_steps.make_future_dataframe(periods=365)   # predict future time series
#    steps_train_predict = model_steps.predict(future)
    
#    future = model_active_duration.make_future_dataframe(periods=365) # predict future time series
#    active_duration_train_predict = model_active_duration.predict(future)
    
#    future = model_active_heart_rate.make_future_dataframe(periods=365)    # predict future time series
#    active_heart_rate_train_predict = model_active_heart_rate.predict(future)
    
#    future = model_distance.make_future_dataframe(periods=365)     # predict future time series
#    distance_train_predict = model_distance.predict(future)
    
#    future = model_calories.make_future_dataframe(periods=365)    # predict future time series
#    calories_train_predict = model_calories.predict(future)
    
#    future = model_resting_heartrate.make_future_dataframe(periods=365)   # predict future time series
#    resting_heartrate_train_predict = model_resting_heartrate.predict(future)
    
#    future = model_sleep.make_future_dataframe(periods=365)    # predict future time series
#    sleep_train_predict = model_sleep.predict(future)
    

    


    

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.


p01


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:

p02


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 14.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 22.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:

...

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_cha

p14


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 19.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:

p15


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 22.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 22.


In [72]:
# write to CSV files

#    steps_train_forcast.to_csv('folder/subfolder/out.csv')
#    active_duration_train_forcast.to_csv('folder/subfolder/out.csv')
#    active_heart_rate_train_forcast.to_csv('folder/subfolder/out.csv')
#    distance_train_forcast.to_csv('folder/subfolder/out.csv')
#    calories_train_forcast.to_csv('folder/subfolder/out.csv')
#    resting_heartrate_train_forcast.to_csv('folder/subfolder/out.csv')
#    sleep_train_forcast.to_csv('folder/subfolder/out.csv')
mape_df.to_csv('Resources/mape_output.csv', index=False)

In [51]:
mape_df

,Participant_ID,steps,active_duration,active_heart_rate,distance,calories,resting_heartrate,sleep,Training_Percentage,Daily_Seasonality
0,p01,74.93,48.43,No Data,58.28,76.53,3.32,18.27,66.0,False


In [13]:
sleep_train_predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-02-09,334.026374,302.386875,404.594661,334.026374,334.026374,18.679239,18.679239,18.679239,18.679239,18.679239,18.679239,0.0,0.0,0.0,352.705614
1,2020-02-10,334.839550,272.716158,376.939979,334.839550,334.839550,-12.632955,-12.632955,-12.632955,-12.632955,-12.632955,-12.632955,0.0,0.0,0.0,322.206595
2,2020-02-11,335.652726,300.983471,403.399997,335.652726,335.652726,15.429111,15.429111,15.429111,15.429111,15.429111,15.429111,0.0,0.0,0.0,351.081836
3,2020-02-12,336.465901,277.112965,379.803573,336.465901,336.465901,-9.258426,-9.258426,-9.258426,-9.258426,-9.258426,-9.258426,0.0,0.0,0.0,327.207475
4,2020-02-13,337.279077,272.540348,379.517325,337.279077,337.279077,-11.070379,-11.070379,-11.070379,-11.070379,-11.070379,-11.070379,0.0,0.0,0.0,326.208698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,2021-03-27,669.054843,653.327606,760.262116,669.054315,669.055327,34.877709,34.877709,34.877709,34.877709,34.877709,34.877709,0.0,0.0,0.0,703.932552
412,2021-03-28,669.868019,639.319563,739.610576,669.867488,669.868506,18.679239,18.679239,18.679239,18.679239,18.679239,18.679239,0.0,0.0,0.0,688.547258
413,2021-03-29,670.681195,602.331598,707.875962,670.680661,670.681685,-12.632955,-12.632955,-12.632955,-12.632955,-12.632955,-12.632955,0.0,0.0,0.0,658.048240
414,2021-03-30,671.494371,634.656927,742.570215,671.493835,671.494864,15.429111,15.429111,15.429111,15.429111,15.429111,15.429111,0.0,0.0,0.0,686.923481


In [14]:
sleep_train_predict = sleep_train_predict.drop(["trend"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["yhat_lower"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["yhat_upper"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["trend_lower"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["trend_upper"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["additive_terms"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["additive_terms_lower"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["additive_terms_upper"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["weekly"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["weekly_lower"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["weekly_upper"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["multiplicative_terms"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["multiplicative_terms_lower"],axis=1)
sleep_train_predict = sleep_train_predict.drop(["multiplicative_terms_upper"],axis=1)


In [15]:
sleep_train_predict

,ds,yhat
0,2020-02-09,352.705614
1,2020-02-10,322.206595
2,2020-02-11,351.081836
3,2020-02-12,327.207475
4,2020-02-13,326.208698
...,...,...
411,2021-03-27,703.932552
412,2021-03-28,688.547258
413,2021-03-29,658.048240
414,2021-03-30,686.923481


In [17]:
sleep_train_predict = sleep_train_predict.rename(columns = {'ds':'Dates', 'yhat':'Minutes_Asleep'}, inplace = True) 
#df.rename(columns = {'old_col1':'new_col1', 'old_col2':'new_col2'}, inplace = True)

AttributeError: 'list' object has no attribute 'rename'

In [24]:
df = pd.DataFrame(sleep_train_predict)

In [25]:
df = df.rename(columns = {'ds':'Dates', 'yhat':'Minutes_Asleep'}, inplace = True)

In [20]:
print(id)

p01


In [26]:
print(df)

None


In [21]:
sleep_forcast_df = sleep_forcast_df.append({'Participant_ID' : id,
                    'Dates' : df['Dates'],
                    'Minutes_Asleep' : df['Minutes_Asleep']},
                    ignore_index=True)

TypeError: 'NoneType' object is not subscriptable

In [38]:
df_sleep.head()

,S.No.,participant_id,dates,minutes_asleep
0,1,p01,2019-11-02,378
1,2,p01,2019-11-03,378
2,3,p01,2019-11-04,361
3,4,p01,2019-11-05,326
4,5,p01,2019-11-06,341
